In [39]:
from processing import convert_to_mne
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

matplotlib.use('Qt5Agg')

path = '../runs/user_2/test_UTENTE_EEG_0/session.csv'

rescale = 1e6
fs = 250
chs = ["Fz", "C3", "Cz", "C4", "Pz", "PO7", "Oz", "PO8"]
columns = ["Fz", "C3", "Cz", "C4", "Pz", "PO7", "Oz", "PO8", "AccX", "AccY", "AccZ", "GyroX", "GyroY", "GyroZ",
           "CNT", "BAT", "VALID", "DeltaTime", "Trigger"]

if 'recordings' in path:
    delimiter = ','
else:
    delimiter = '\t'

data = pd.read_csv(filepath_or_buffer=path, delimiter=delimiter)
data.columns = columns[:data.shape[1]-1] + [columns[-1]]

print(data.head())

minutes = len(data)/(60*fs)
seconds = (len(data)/fs) % 60 
print(f'Recording duration: {int(minutes)} minutes and {int(seconds)} seconds')

trigger = data.iloc[:, -1].to_numpy(dtype=np.float64)
print(np.unique(trigger))

eeg = data.iloc[:, 0:8].to_numpy(dtype=np.float64)

raw_data = convert_to_mne(eeg, trigger, rescale=rescale, fs=fs, chs=chs, recompute=False)

              Fz           C3             Cz             C4           Pz  \
0  105024.859375  549743.1250    6883.264648 -135281.703125  569073.5000   
1 -103244.593750  502041.7500  -35517.011719 -116807.898438  558928.8125   
2  117869.781250  582385.5625  143297.312500  -31326.507812  712523.3125   
3  299057.250000  668572.8750  189050.968750 -152220.218750  708806.7500   
4  356560.687500  698277.1875  136229.687500  316050.468750  712238.4375   

             PO7             Oz            PO8      AccX      AccY      AccZ  \
0  179174.718750 -603728.062500  623638.625000  0.061279  1.030029 -0.094971   
1  159696.609375  -48527.250000  374406.593750  0.061523  1.029541 -0.100586   
2  191211.140625      18.596653  263378.125000  0.061279  1.030029 -0.103271   
3  327121.468750     185.251266  127928.960938  0.062744  1.029053 -0.105225   
4  634246.500000   30925.070312  267686.468750  0.062256  1.027100 -0.109131   

      GyroX    GyroY     GyroZ        CNT  BAT  VALID     Delt

In [40]:
# Compute PSD
Pxx = raw_data.compute_psd(fmin=0, fmax=fs/2)
Pxx.plot()
plt.show()

Effective window size : 8.192 (s)


In [41]:
filtered = raw_data.copy() # the method filters the signal in-place, so this time I
                      # want to preserve the original signal and filter just a
                      # temporary copy of it

# remove power line noise
filtered.notch_filter(50) 
filtered.notch_filter(60) 
# Apply band-pass filtering
filtered.filter(1,30) 

pxx_filt = filtered.compute_psd(fmin=0, fmax=50)
pxx_filt.plot()
plt.show()

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband e

In [42]:
filtered.plot()
plt.show()

Channels marked as bad:
none


### Channels Interpolation
Previously marked bad channels can be interpolated

In [32]:
## Interpolate bad channels
interp = filtered.copy()

# Mark the bad channels in the raw data
print(interp.info['bads'])

interp.interpolate_bads()
interp.plot()
plt.show()

[]
Setting channel interpolation method to {'eeg': 'spline'}.


C:\Users\Gianni\AppData\Local\Temp\ipykernel_10712\2769900535.py:7: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  interp.interpolate_bads()


Channels marked as bad:
none


### Cleaning data using ASR
Full example available: https://github.com/DiGyt/asrpy/blob/main/example.ipynb

In [33]:
## take the first 30 seconds of the data
training_data = interp.copy()
training_data.crop(tmin=5, tmax=65)
training_data.plot()
plt.show()

Channels marked as bad:
none


In [35]:
import asrpy

asr = asrpy.ASR(sfreq=fs, cutoff=15)
asr.fit(training_data)
cleaned = asr.transform(interp)
cleaned.plot()
plt.show()

Channels marked as bad:
none
